In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_parquet('../data/yellow_tripdata_2023-01.parquet')

In [3]:
df.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
0,2,2023-01-01 00:32:10,2023-01-01 00:40:36,1.0,0.97,1.0,N,161,141,2,9.3,1.00,0.5,0.00,0.0,1.0,14.30,2.5,0.00
1,2,2023-01-01 00:55:08,2023-01-01 01:01:27,1.0,1.10,1.0,N,43,237,1,7.9,1.00,0.5,4.00,0.0,1.0,16.90,2.5,0.00
2,2,2023-01-01 00:25:04,2023-01-01 00:37:49,1.0,2.51,1.0,N,48,238,1,14.9,1.00,0.5,15.00,0.0,1.0,34.90,2.5,0.00
3,1,2023-01-01 00:03:48,2023-01-01 00:13:25,0.0,1.90,1.0,N,138,7,1,12.1,7.25,0.5,0.00,0.0,1.0,20.85,0.0,1.25
4,2,2023-01-01 00:10:29,2023-01-01 00:21:19,1.0,1.43,1.0,N,107,79,1,11.4,1.00,0.5,3.28,0.0,1.0,19.68,2.5,0.00


In [4]:
df.dtypes

VendorID                          int64
tpep_pickup_datetime     datetime64[us]
tpep_dropoff_datetime    datetime64[us]
passenger_count                 float64
trip_distance                   float64
RatecodeID                      float64
store_and_fwd_flag               object
PULocationID                      int64
DOLocationID                      int64
payment_type                      int64
fare_amount                     float64
extra                           float64
mta_tax                         float64
tip_amount                      float64
tolls_amount                    float64
improvement_surcharge           float64
total_amount                    float64
congestion_surcharge            float64
airport_fee                     float64
dtype: object

In [5]:
f'{df.shape[0]:,}'

'3,066,766'

In [6]:
df.shape

(3066766, 19)

In [7]:
df['duration'] = df['tpep_dropoff_datetime'] - df['tpep_pickup_datetime']
df['duration'] = df['duration'].apply(lambda x: round(x.total_seconds()/60,3))

In [8]:
df.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee,duration
0,2,2023-01-01 00:32:10,2023-01-01 00:40:36,1.0,0.97,1.0,N,161,141,2,9.3,1.00,0.5,0.00,0.0,1.0,14.30,2.5,0.00,8.433
1,2,2023-01-01 00:55:08,2023-01-01 01:01:27,1.0,1.10,1.0,N,43,237,1,7.9,1.00,0.5,4.00,0.0,1.0,16.90,2.5,0.00,6.317
2,2,2023-01-01 00:25:04,2023-01-01 00:37:49,1.0,2.51,1.0,N,48,238,1,14.9,1.00,0.5,15.00,0.0,1.0,34.90,2.5,0.00,12.750
3,1,2023-01-01 00:03:48,2023-01-01 00:13:25,0.0,1.90,1.0,N,138,7,1,12.1,7.25,0.5,0.00,0.0,1.0,20.85,0.0,1.25,9.617
4,2,2023-01-01 00:10:29,2023-01-01 00:21:19,1.0,1.43,1.0,N,107,79,1,11.4,1.00,0.5,3.28,0.0,1.0,19.68,2.5,0.00,10.833


In [9]:
df.duration.describe()

count    3.066766e+06
mean     1.566900e+01
std      4.259435e+01
min     -2.920000e+01
25%      7.117000e+00
50%      1.151700e+01
75%      1.830000e+01
max      1.002918e+04
Name: duration, dtype: float64

In [10]:
df_new = df[(df.duration >= 1) & (df.duration <= 60)]

In [11]:
df_new.shape[0]/df.shape[0]     # Percentage of rows remaining 

0.9812202822125979

In [12]:
features = ['PULocationID','DOLocationID']

In [13]:
df_new[features] = df_new[features].astype(str)

C:\Users\Lenovo\AppData\Local\Temp\ipykernel_23764\3574605497.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_new[features] = df_new[features].astype(str)


## Pre-process and Train

In [15]:
train_dicts = df_new[features].to_dict(orient = 'records')

In [20]:
train_dicts[:5]      # Convert dataframe into dictionary of key and values for each row

[{'PULocationID': '161', 'DOLocationID': '141'},
 {'PULocationID': '43', 'DOLocationID': '237'},
 {'PULocationID': '48', 'DOLocationID': '238'},
 {'PULocationID': '138', 'DOLocationID': '7'},
 {'PULocationID': '107', 'DOLocationID': '79'}]

In [22]:
from sklearn.feature_extraction import DictVectorizer

In [23]:
from sklearn.metrics import mean_squared_error as mse

In [24]:
dv = DictVectorizer()    # Just a new way of one-hot encoding categorical values and save sparse storage 
x_train = dv.fit_transform(train_dicts)

In [25]:
x_train

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 6018346 stored elements and shape (3009173, 515)>

In [26]:
y_train = df_new['duration']

In [27]:
len(dv.feature_names_)

515

In [28]:
from sklearn.linear_model import LinearRegression

In [29]:
lr = LinearRegression()

In [30]:
lr.fit(x_train, y_train)

LinearRegression()

## Validate

In [31]:
"""
    Description: Read data from file location and save it in a dataframe afer applying filtering
    Input: Parquet file location as string
    Output: Filtered dataframe
"""

def read_data(location):
    df = pd.read_parquet(location)
    df['duration'] = df['tpep_dropoff_datetime'] - df['tpep_pickup_datetime']
    df['duration'] = df['duration'].apply(lambda x: round(x.total_seconds()/60,3))
    return df[(df.duration >= 1) & (df.duration <= 60)]

In [32]:

"""
    Input: dataframe and DictVectorizer to transform
    Output: tuple containing features, labels, feature_names from the 
    note: 
        features are in compressed sparse row format (which is generated using sklearn.DictVectorizer) 
"""

def prepare_data(data, dv):
    features = ['PULocationID','DOLocationID']
    data[features] = data[features].astype(str)
    feature_dict = data[features].to_dict(orient = 'records')
    features_vectors = dv.transform(feature_dict)         # To preserve the same features after one-hot-encoding
    labels = data['duration']
    return features_vectors, labels, dv.feature_names_

In [33]:
val_df = read_data('../data/yellow_tripdata_2023-02.parquet')

In [34]:
features, labels, feature_names = prepare_data(val_df, dv)

In [35]:
features

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 5711894 stored elements and shape (2855951, 515)>

In [36]:
labels[:5]

0     1.683
3    32.083
4    13.300
5    14.633
6    27.950
Name: duration, dtype: float64

In [37]:
from sklearn.metrics import mean_squared_error
import math

In [38]:
# Training Loss
y_pred = lr.predict(x_train)
print(math.sqrt(mean_squared_error(y_train, y_pred)))

7.649261986883684


In [39]:

# Validation Loss
y_pred = lr.predict(features)
print(math.sqrt(mean_squared_error(labels, y_pred)))

7.811817555153357


In [40]:
len(feature_names)

515

In [41]:
set(dv.feature_names_).difference(set(feature_names))

set()

In [42]:
set(feature_names).difference(set(dv.feature_names_))

set()

In [43]:
df_new.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee,duration
0,2,2023-01-01 00:32:10,2023-01-01 00:40:36,1.0,0.97,1.0,N,161,141,2,9.3,1.00,0.5,0.00,0.0,1.0,14.30,2.5,0.00,8.433
1,2,2023-01-01 00:55:08,2023-01-01 01:01:27,1.0,1.10,1.0,N,43,237,1,7.9,1.00,0.5,4.00,0.0,1.0,16.90,2.5,0.00,6.317
2,2,2023-01-01 00:25:04,2023-01-01 00:37:49,1.0,2.51,1.0,N,48,238,1,14.9,1.00,0.5,15.00,0.0,1.0,34.90,2.5,0.00,12.750
3,1,2023-01-01 00:03:48,2023-01-01 00:13:25,0.0,1.90,1.0,N,138,7,1,12.1,7.25,0.5,0.00,0.0,1.0,20.85,0.0,1.25,9.617
4,2,2023-01-01 00:10:29,2023-01-01 00:21:19,1.0,1.43,1.0,N,107,79,1,11.4,1.00,0.5,3.28,0.0,1.0,19.68,2.5,0.00,10.833
